# pandas ecosystem 2019

@datapythonista

## About me

Marc Garcia

**@datapythonista** (Twitter, GitHub, LinkedIn,...)

- pandas core dev
- Python fellow
- 13 years of experience with Python
- Contractor data scientist

![](img/pandas_ecosystem.jpg)

## Agenda

- pandas components
- hardware and numpy
- data structures
- indexing
- functionality
- pandas API
- pandas alternatives
- distributing pandas

![](img/software.png)

![](img/components_01_numpy.png)

In [1]:
2 + 2

4

In [2]:
class Object:
    def __add__(self, other):
        # add numbers
        # contactenate strings or arrays
        # add date to delta
        return do_something(self, other)

**PyObject**

- type
- reference count (garbage collection)
- value
  - can be a complex structure (e.g. arbitrary-precision integers)

![](img/cpu_memory_speed.jpg)

Python lists at a scale (e.g. adding 1,000,000 integers)

- Not very efficient **representation** if we can assume homogenous types
- **Transfering** a lot of extra information from memory to CPU
- Poor use of CPU **caches**
- **Slow** compared to operations with homogenous types in C

More efficient representation: `array` module

In [10]:
import array

array.array('I', [1, 2, 3, 4])

array('I', [1, 2, 3, 4])

In [14]:
import random

big_list = [random.randint(0, 255) for i in range(10_000_000)]

big_array = array.array('I', big_list)

In [15]:
%timeit sum(big_list)

47.7 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit sum(big_array)

58.2 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
import numpy

big_numpy = numpy.array(big_array)

In [19]:
%timeit big_numpy.sum()

7.77 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
numpy.array([(1, 2, 3),
             (4, 5, 6)],
            dtype=numpy.uint8)

array([[1, 2, 3],
       [4, 5, 6]], dtype=uint8)

![](img/components_02_data_structures.png)

In [21]:
python_list = [1, 2, 3, 4]

python_list[2]

3

In [22]:
python_list[1:-1]

[2, 3]

In [23]:
python_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 4}

python_dict['b']

2

In [27]:
import pandas

series = pandas.Series([1, 2, 3, 4],
                       index=['a', 'b', 'c', 'd'])

In [28]:
series[1:-1]

b    2
c    3
dtype: int64

In [29]:
series['b']

2

In [31]:
series.dtype

dtype('int64')

In [32]:
series + 1

a    2
b    3
c    4
d    5
dtype: int64

In [34]:
series.mean()

2.5

In [41]:
series.values

array([1, 2, 3, 4])

In [47]:
df = pandas.DataFrame({'countries': ['it', 'pt', 'es', 'gr'],
                       'numbers': series,
                       'floats': [3.141592, 2.718281828, float('NaN'), 0.]})
df

,countries,numbers,floats
a,it,1,3.141592
b,pt,2,2.718282
c,es,3,NaN
d,gr,4,0.000000


![](img/arrow_web.png)

![](img/apache_memory.png)

**pandas backend requirements:**
    
1. efficient data representation
2. fast memory access
3. fast operations (e.g. sum)

Apache Arrow is mature in 1 and 2. Work still needed for 3 (Gandiva is an effort in this way).

Wes is hiring full-time Apache Arrow developers (C++). Check Ursa Labs.

![](img/components_arrow.png)

![](img/components_backends.png)

In practice: **Extension arrays**

- Backend still mainly based on numpy
- Columns can use other libraries
  - Example: **Fletcher** (Arrow strings with the current backend)

![](img/components_03_indexing.png)

**Labels**

- Key feature of pandas
- Access data by names (rows or columns)
- Support of multiindices

![](img/stuck_pandas.jpg)

![](img/pandas_indexing.png)

In [6]:
gdp = (pandas.DataFrame([('USA', 'Americas', 19_390_604, 20_035_183, 47., 49.),
                        ('China', 'Asia', 12_237_700, 15_554_902, 46.5, 48.),
                        ('Japan', 'Asia', 4_872_137, 4_893_502, 37.9, 36.2),
                        ('Germany', 'Europe', 3_677_439, 3_732_192, 27., 27.3),
                        ('UK', 'Europe', 2_622_434, 2_591_883, 32.4, 34.1),
                        ('India', 'Asia', 2_597_491, 2_712_658, 35.1, 37.9)],
                       columns=['country', 'continent', None, None, None, None])
             .set_index(['continent', 'country']))
gdp.columns = [['gdp', 'gdp', 'gini', 'gini'], [2017, 2018] * 2]
gdp

gdp            gini      
                       2017      2018  2017  2018
continent country                                
Americas  USA      19390604  20035183  47.0  49.0
Asia      China    12237700  15554902  46.5  48.0
          Japan     4872137   4893502  37.9  36.2
Europe    Germany   3677439   3732192  27.0  27.3
          UK        2622434   2591883  32.4  34.1
Asia      India     2597491   2712658  35.1  37.9

In [60]:
gdp.loc['Asia']

gdp            gini      
             2017      2018  2017  2018
country                                
China    12237700  15554902  46.5  48.0
Japan     4872137   4893502  37.9  36.2
India     2597491   2712658  35.1  37.9

In [61]:
gdp.loc['Asia', 'gdp']

,2017,2018
country,,
China,12237700,15554902
Japan,4872137,4893502
India,2597491,2712658


In [63]:
gdp.xs(2018, axis='columns', level=1)

gdp  gini
continent country                
Americas  USA      20035183  49.0
Asia      China    15554902  48.0
          Japan     4893502  36.2
Europe    Germany   3732192  27.3
          UK        2591883  34.1
Asia      India     2712658  37.9

pandas indexing system is reused by other projects, mainly **xarray**

xarray data structures are N-dimensional labelled arrays (based on numpy).

![](img/xarray.png)

In [65]:
import xarray

gdp_xarray = xarray.Dataset.from_dataframe(gdp)
gdp_xarray

<xarray.Dataset>
Dimensions:         (continent: 3, country: 6)
Coordinates:
  * continent       (continent) object 'Americas' 'Asia' 'Europe'
  * country         (country) object 'China' 'Germany' 'India' ... 'UK' 'USA'
Data variables:
    ('gdp', 2017)   (continent, country) float64 nan nan nan ... 2.622e+06 nan
    ('gdp', 2018)   (continent, country) float64 nan nan nan ... 2.592e+06 nan
    ('gini', 2017)  (continent, country) float64 nan nan nan ... nan 32.4 nan
    ('gini', 2018)  (continent, country) float64 nan nan nan ... nan 34.1 nan

In [69]:
gdp_xarray.loc[{'continent': 'Asia'}]

<xarray.Dataset>
Dimensions:         (country: 6)
Coordinates:
    continent       <U4 'Asia'
  * country         (country) object 'China' 'Germany' 'India' ... 'UK' 'USA'
Data variables:
    ('gdp', 2017)   (country) float64 1.224e+07 nan 2.597e+06 4.872e+06 nan nan
    ('gdp', 2018)   (country) float64 1.555e+07 nan 2.713e+06 4.894e+06 nan nan
    ('gini', 2017)  (country) float64 46.5 nan 35.1 37.9 nan nan
    ('gini', 2018)  (country) float64 48.0 nan 37.9 36.2 nan nan

## pandas functionality

![](img/components_04_io.png)

<table class="colwidths-given docutils" border="1">
<colgroup>
<col width="12%">
<col width="40%">
<col width="24%">
<col width="24%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Format Type</th>
<th class="head">Data Description</th>
<th class="head">Reader</th>
<th class="head">Writer</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>text</td>
<td><a class="reference external" href="https://en.wikipedia.org/wiki/Comma-separated_values">CSV</a></td>
<td><a class="reference internal" href="#io-read-csv-table"><span class="std std-ref">read_csv</span></a></td>
<td><a class="reference internal" href="#io-store-in-csv"><span class="std std-ref">to_csv</span></a></td>
</tr>
<tr class="row-odd"><td>text</td>
<td><a class="reference external" href="https://www.json.org/">JSON</a></td>
<td><a class="reference internal" href="#io-json-reader"><span class="std std-ref">read_json</span></a></td>
<td><a class="reference internal" href="#io-json-writer"><span class="std std-ref">to_json</span></a></td>
</tr>
<tr class="row-even"><td>text</td>
<td><a class="reference external" href="https://en.wikipedia.org/wiki/HTML">HTML</a></td>
<td><a class="reference internal" href="#io-read-html"><span class="std std-ref">read_html</span></a></td>
<td><a class="reference internal" href="#io-html"><span class="std std-ref">to_html</span></a></td>
</tr>
<tr class="row-odd"><td>text</td>
<td>Local clipboard</td>
<td><a class="reference internal" href="#io-clipboard"><span class="std std-ref">read_clipboard</span></a></td>
<td><a class="reference internal" href="#io-clipboard"><span class="std std-ref">to_clipboard</span></a></td>
</tr>
<tr class="row-even"><td>binary</td>
<td><a class="reference external" href="https://en.wikipedia.org/wiki/Microsoft_Excel">MS Excel</a></td>
<td><a class="reference internal" href="#io-excel-reader"><span class="std std-ref">read_excel</span></a></td>
<td><a class="reference internal" href="#io-excel-writer"><span class="std std-ref">to_excel</span></a></td>
</tr>
<tr class="row-odd"><td>binary</td>
<td><a class="reference external" href="https://support.hdfgroup.org/HDF5/whatishdf5.html">HDF5 Format</a></td>
<td><a class="reference internal" href="#io-hdf5"><span class="std std-ref">read_hdf</span></a></td>
<td><a class="reference internal" href="#io-hdf5"><span class="std std-ref">to_hdf</span></a></td>
</tr>
<tr class="row-even"><td>binary</td>
<td><a class="reference external" href="https://github.com/wesm/feather">Feather Format</a></td>
<td><a class="reference internal" href="#io-feather"><span class="std std-ref">read_feather</span></a></td>
<td><a class="reference internal" href="#io-feather"><span class="std std-ref">to_feather</span></a></td>
</tr>
<tr class="row-odd"><td>binary</td>
<td><a class="reference external" href="https://parquet.apache.org/">Parquet Format</a></td>
<td><a class="reference internal" href="#io-parquet"><span class="std std-ref">read_parquet</span></a></td>
<td><a class="reference internal" href="#io-parquet"><span class="std std-ref">to_parquet</span></a></td>
</tr>
<tr class="row-even"><td>binary</td>
<td><a class="reference external" href="https://msgpack.org/index.html">Msgpack</a></td>
<td><a class="reference internal" href="#io-msgpack"><span class="std std-ref">read_msgpack</span></a></td>
<td><a class="reference internal" href="#io-msgpack"><span class="std std-ref">to_msgpack</span></a></td>
</tr>
<tr class="row-odd"><td>binary</td>
<td><a class="reference external" href="https://en.wikipedia.org/wiki/Stata">Stata</a></td>
<td><a class="reference internal" href="#io-stata-reader"><span class="std std-ref">read_stata</span></a></td>
<td><a class="reference internal" href="#io-stata-writer"><span class="std std-ref">to_stata</span></a></td>
</tr>
<tr class="row-even"><td>binary</td>
<td><a class="reference external" href="https://en.wikipedia.org/wiki/SAS_(software)">SAS</a></td>
<td><a class="reference internal" href="#io-sas-reader"><span class="std std-ref">read_sas</span></a></td>
<td>&nbsp;</td>
</tr>
<tr class="row-odd"><td>binary</td>
<td><a class="reference external" href="https://docs.python.org/3/library/pickle.html">Python Pickle Format</a></td>
<td><a class="reference internal" href="#io-pickle"><span class="std std-ref">read_pickle</span></a></td>
<td><a class="reference internal" href="#io-pickle"><span class="std std-ref">to_pickle</span></a></td>
</tr>
<tr class="row-even"><td>SQL</td>
<td><a class="reference external" href="https://en.wikipedia.org/wiki/SQL">SQL</a></td>
<td><a class="reference internal" href="#io-sql"><span class="std std-ref">read_sql</span></a></td>
<td><a class="reference internal" href="#io-sql"><span class="std std-ref">to_sql</span></a></td>
</tr>
<tr class="row-odd"><td>SQL</td>
<td><a class="reference external" href="https://en.wikipedia.org/wiki/BigQuery">Google Big Query</a></td>
<td><a class="reference internal" href="#io-bigquery"><span class="std std-ref">read_gbq</span></a></td>
<td><a class="reference internal" href="#io-bigquery"><span class="std std-ref">to_gbq</span></a></td>
</tr>
</tbody>
</table>

In [4]:
import pandas

countries = pandas.read_csv('https://github.com/datapythonista/pandas_ecosystem/raw/master/data/countries.csv.gz',
                            sep=';')
countries

,Country (en),Country (de),Country (local),Country code,Continent,Capital,Population,Area,Coastline,Government form,Currency,Currency code,Dialing prefix,Birthrate,Deathrate,Url
0,Albania,Albanien,Shqipëria,AL,Europe,Tirana,2873457,28750,362,Parliamentary republic,Lek,ALL,355,11.8,7.4,https://www.laenderdaten.info/Europa/Albanien/...
1,Angola,Angola,Angola,AO,Africa,Luanda,29784193,1246700,1600,Presidential republic,Kwanza,AOA,244,41.8,8.5,https://www.laenderdaten.info/Afrika/Angola/in...
2,Equatorial Guinea,Äquatorialguinea,Guinea Ecuatorial,GQ,Africa,Malabo,1267689,28050,296,Presidential republic,Central African Franc,XAF,240,34.1,10.2,https://www.laenderdaten.info/Afrika/Aequatori...
3,Azerbaijan,Aserbaidschan,Azärbaycan,AZ,Asia,Baku,9862429,86600,0,Presidential republic,Manat,AZN,994,16.3,5.8,https://www.laenderdaten.info/Asien/Aserbaidsc...


![](img/components_05_joins.png)

**Joins**

![](img/pandas_join.png)

**Concatenate**

![](img/pandas_concat.png)

![](img/components_06_reshape.png)

![](img/pandas_reshape.png)

![](img/components_07_groupby.png)

In [11]:
gdp.groupby('continent').mean()

gdp                   gini      
                   2017        2018       2017  2018
continent                                           
Americas   1.939060e+07  20035183.0  47.000000  49.0
Asia       6.569109e+06   7720354.0  39.833333  40.7
Europe     3.149936e+06   3162037.5  29.700000  30.7

In [12]:
gdp.groupby(axis='columns', level=1).mean()

2017         2018
continent country                         
Americas  USA      9695325.50  10017616.00
Asia      China    6118873.25   7777475.00
          Japan    2436087.45   2446769.10
Europe    Germany  1838733.00   1866109.65
          UK       1311233.20   1295958.55
Asia      India    1298763.05   1356347.95

![](img/components_08_window.png)

`.rolling()` (e.g. moving average)

![](img/moving_average.png)

![](img/components_09_stats.png)

In [13]:
gdp.describe()

gdp                     gini          
               2017          2018       2017      2018
count  6.000000e+00  6.000000e+00   6.000000   6.00000
mean   7.566301e+06  8.253387e+06  37.650000  38.75000
std    6.828598e+06  7.571066e+06   7.915744   8.37347
min    2.597491e+06  2.591883e+06  27.000000  27.30000
25%    2.886185e+06  2.967542e+06  33.075000  34.62500
50%    4.274788e+06  4.312847e+06  36.500000  37.05000
75%    1.039631e+07  1.288955e+07  44.350000  45.47500
max    1.939060e+07  2.003518e+07  47.000000  49.00000